# Azure Machine Learning のセットアップ
まず、Azureサブスクリプションに関する次の情報を入力する必要があります。

**独自のAzureサブスクリプションを使用している場合は、使用するsubscription_id、resource_group、workspace_name、workspace_regionの名前を指定してください。** ワークスペースのタイプは [Machine Learning Workspace](https://docs.microsoft.com/en-us/azure/machine-learning/service/setup-create-workspace) である必要があることに注意してください。

**環境が提供されている場合は、以下の値のXXXXXを一意の識別子に置き換えてください。**

次のセルで、コメントの指示に従って、 `subscription_id`、` resource_group`、 `workspace_name`、および` workspace_region`の値を必ず設定してください (*これらの値は、Azureポータルから取得できます*).

これらの値を取得するには、次の操作を行います:

1. Azureポータルに移動し、提供された資格情報でログインします。

2. 左側のメニューの[お気に入り]で、[リソースグループ]を選択します。

3. リストで、「XXXXX」のような名前のリソースグループを選択します。

4. 「概要」タブから、必要な値を取得します。

上のコマンドバーの `>|Run` ボタンを選択して、次のセルを実行します。

In [ ]:
#既存のAzureサブスクリプションのサブスクリプションIDに置き換える
subscription_id = "" #"<your-azure-subscription-id>"

#Azure ML関連のサービスを含むリソースグループの名前を指定します
resource_group = "mcwailab" #"<your-subscription-group-name>"

#作成されるAzure Machine Learningワークスペースの一意の名前とリージョンを指定します
workspace_name = "mcw-csdl"
workspace_region = "eastus" # japaneast, eastus2, eastus, westcentralus, southeastasia, australiaeast, westeurope

## Azure Machine Learningワークスペースを作成して接続する

Azure Machine Learning Python SDKは、Azure Machine Learningサービスの実験、モデル管理、モデルデプロイメント機能を活用するために必要です。 次のセルを実行して、新しいAzure Machine Learning **Workspace** を作成し、構成をディスクに保存します。 `config.json` という名前の設定ファイルは、` .azureml` という名前のフォルダーに保存されます。

**重要**: セルの下に出力されるテキストでログインするように求められます。 表示されたURLに移動し、提供されているコードを入力してください。コードを入力したら、このノートブックに戻り、 `Workspace configuration succeeded` と出力されるのを待ちます。

**任意**: 複数のテナントを持っている場合、以下のセルをアクティブにしてください。クラスをインポートし、ターゲットとするテナントを明示的に定義する必要があります。 
InteractiveLoginAuthentication のコンストラクターを呼び出すと、指定したテナントIDに基づいてログインするよう求められます。


In [ ]:
#任意: 複数のテナントを持っている場合
#from azureml.core.authentication import InteractiveLoginAuthentication
#auth = InteractiveLoginAuthentication(tenant_id = '<your-tenant-id>')

上記セルをアクティブにした場合、下記の `Workspace.create` メソッドで `auth = auth` パラメータを有効化します。

In [ ]:
import azureml.core
print('azureml.core.VERSION: ', azureml.core.VERSION)

#　Workspaceクラスをインポートしてazureml SDKのバージョンを確認する
from azureml.core import Workspace

ws = Workspace.create(
    name = workspace_name,
    subscription_id = subscription_id,
    resource_group = resource_group, 
    location = workspace_region, 
    exist_ok = True
#   ,auth = auth
    )

ws.write_config()
print('Workspace configuration succeeded')

次のセルを実行して、生成された構成ファイルの内容を確認します。

In [ ]:
!cat .azureml/config.json

# Azure Container Instance（ACI）にモデルをデプロイする

このセクションでは、 `01 Summarize` に示すように、Gensimを使用してテキストを要約するWebサービスをデプロイします。 WebサービスはAzure Container Serviceでホストされます。

## スコアリングWebサービスを作成する

Azure Machine Learningサービスでスコアリング用のモデルをデプロイする場合、モデルをロードしてスコアリングに使用する単純なWebサービスのコードを定義する必要があります。慣例により、このサービスには、モデルをロードするinitと、ロードされたモデルを使用してデータをスコアリングするrunの2つのメソッドがあります。

このスコアリングサービスコードは、後で特別に準備されたDockerコンテナー内にデプロイされます。

In [ ]:
%%writefile summarizer_service.py

import re
import nltk
import unicodedata
from gensim.summarization import summarize, keywords

def clean_and_parse_document(document):
    if isinstance(document, str):
        document = document
    elif isinstance(document, unicode):
        return unicodedata.normalize('NFKD', document).encode('ascii', 'ignore')
    else:
        raise ValueError("Document is not string or unicode.")
    document = document.strip()
    sentences = nltk.sent_tokenize(document)
    sentences = [sentence.strip() for sentence in sentences]
    return sentences

def summarize_text(text, summary_ratio=None, word_count=30):
    sentences = clean_and_parse_document(text)
    cleaned_text = ' '.join(sentences)
    summary = summarize(cleaned_text, split=True, ratio=summary_ratio, word_count=word_count)
    return summary 

def init():  
    nltk.download('all')
    return

def run(input_str):
    try:
        return summarize_text(input_str)
    except Exception as e:
        return (str(e))

## 環境

Azure ML環境は、機械学習トレーニングが行われる環境をカプセル化したものです。これらは、Pythonパッケージ、環境変数、Docker設定、およびその他の属性を宣言型で定義します。環境はバージョン管理されています。環境を更新し、古いバージョンを取得して、作業を再検討および確認できます。

環境により、次のことが可能になります:
* Pythonパッケージやそのバージョンなど、トレーニングプロセスの依存関係をカプセル化します
* VMまたはMLComputeクラスターでのリモート実行でローカルコンピューター上のPython環境を再現します
* 実稼働環境で実験環境を再現します
* 既存のモデルがトレーニングされた環境を再検討して監査します

環境、計算ターゲット、トレーニングスクリプトが一緒になって、実行構成を形成します。

### キュレーションされた環境を利用する

キュレーションされた環境はAzure Machine Learningによって提供され、デフォルトでワークスペースで利用できます。これらの環境には、さまざまな機械学習フレームワークを始めるのに役立つ Python パッケージと設定のコレクションが含まれています。

  * AzureML-Minimal__環境には、ランのトラッキングとアセットのアップロードを可能にする最小限のパッケージが含まれています。ご自身の環境の出発点として使用することができます。
  * AzureML-Tutorial__環境には、Scikit-Learn、Pandas、Matplotlibなどの一般的なデータサイエンスパッケージと、azureml-sdkパッケージの大規模なセットが含まれています。

キュレーションされた環境は、キャッシュされたDockerイメージによってバックアップされ、ラン準備コストを削減します。

詳細は https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/training/using-environments/using-environments.ipynb を参照してください。

### 自分の環境を作る

環境をキュレーションする代わりに、```Environment```オブジェクトをインスタンス化して環境を作成し、その属性(Pythonパッケージのセット、環境変数など)を設定します。このトレーニングでは、このアプローチを採用します。

#### Pythonのパッケージを追加

推奨される方法は、Conda パッケージを指定することです。また、pip パッケージを追加して、パッケージのバージョンを指定することもできます。

In [ ]:
from azureml.core import Environment
from azureml.core.environment import CondaDependencies

myenv = Environment(name="myenv")
conda_dep = CondaDependencies()
conda_dep.add_pip_package("gensim")
conda_dep.add_pip_package("nltk")
myenv.python.conda_dependencies=conda_dep

### 環境の登録

環境を登録することで、環境を管理することができます。これにより、環境のバージョンを追跡し、将来の実験で再利用することができます。例えば、要件を満たす環境を構築したら、その環境を登録して他の実験で使用することで、ワークフローの標準化を図ることができます。

同じ名前で環境を登録すると、バージョン番号が1つ増えます。なお、Azure MLではバージョン間の差分を記録しているので、同じバージョンを再登録してもバージョン番号は増えません。

In [ ]:
myenv.register(workspace=ws)

## デプロイ

モデルの実行方法をより制御したい場合、別のフレームワークを使用している場合、または特別なランタイム要件がある場合は、独自の環境とスコアリング方法を指定することができます。カスタム環境は、デプロイしたいモデルに使用することができます。

前のコードでは、[Environment](https://docs.microsoft.com/python/api/azureml-core/azureml.core.environment%28class%29?view=azure-ml-py)オブジェクトを作成し、モデルに必要な[CondaDependencies](https://docs.microsoft.com/python/api/azureml-core/azureml.core.conda_dependencies.condadependencies?view=azure-ml-py)を提供することで、モデルの実行環境を指定しました。

次のセルでは、Azure Machine Learning SDK を使用して、モデルとスコアリングスクリプトをコンテナにパッケージ化し、そのコンテナを Azure コンテナインスタンスにデプロイします。

以下のセルを実行します。

In [ ]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

inference_config = InferenceConfig(entry_script='summarizer_service.py', environment=myenv)
aci_config = AciWebservice.deploy_configuration(
    cpu_cores = 1, 
    memory_gb = 1, 
    tags = {'name':'Summarization'}, 
    description = 'Summarizes text.')

これで、Azureコンテナインスタンスへのデプロイを開始する準備が整いました。 この場合、モデルはPythonスクリプトの中にあるので、_model_パラメータには空白のリストがあります。

次のセルを実行してください: _Running_タグで進行状況のドットが追加されている間、完了まで5～15分ほどかかるかもしれません。

Webサービスの準備ができたら、以下のような出力が表示されます。

`
Succeeded
ACI service creation operation finished, operation "Succeeded"`

In [ ]:
from azureml.core import Model
from azureml.core import Webservice
from azureml.exceptions import WebserviceException

service_name = "summarizer"

# Remove any existing service under the same name.
try:
    Webservice(ws, service_name).delete()
except WebserviceException:
    pass

webservice = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[],
                       inference_config=inference_config,
                       deployment_config=aci_config)
webservice.wait_for_deployment(show_output=True)

## デプロイされたサービスをテストする

これで、デプロイされたWebサービスを使用してスコアリングをテストする準備ができました。 次のセルはWebサービスを呼び出します。

次のセルを実行して、デプロイされたWebサービスに対して単一の入力行を使用してスコアリングをテストします。

In [ ]:
example_document = """
I was driving down El Camino and stopped at a red light.
It was about 3pm in the afternoon.  
The sun was bright and shining just behind the stoplight.
This made it hard to see the lights.
There was a car on my left in the left turn lane.
A few moments later another car, a black sedan pulled up behind me. 
When the left turn light changed green, the black sedan hit me thinking 
that the light had changed for us, but I had not moved because the light 
was still red.
After hitting my car, the black sedan backed up and then sped past me.
I did manage to catch its license plate. 
The license plate of the black sedan was ABC123. 
"""

In [ ]:
result = webservice.run(input_data = example_document)
print(result)

## スコアリングURIをキャプチャする

RESTクライアントからサービスを呼び出すには、スコアリングURIを取得する必要があります。 次のセルを実行してスコアリングURIを取得し、この値をメモします。最後のノートブックで必要になります。

In [ ]:
webservice.scoring_uri

このサービスのデプロイに使用されるデフォルト設定では、認証を必要としないサービスが作成されるため、このサービスを呼び出すために必要な値はスコアリングURIのみです。